In [33]:
import os
import requests
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool, Tool
from pydantic import BaseModel, Field
from langchain.tools import BaseTool
from typing import Any, Type
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.utilities import WikipediaAPIWrapper
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents import create_sql_agent
from fake_useragent import UserAgent

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", 
    temperature=0.1,
    )


In [34]:
class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class DuckDuckGoSearchTool(BaseTool):
    name: str = "DuckDuckGoSearchTool"
    description: str = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ua = UserAgent()
        custom_user_agent = ua.random
        #ddg = DuckDuckGoSearchAPIWrapper()
        ddg = DuckDuckGoSearchRun(user_agent=custom_user_agent)
        return ddg.run(query)


class WikipediaSearchTool(BaseTool):
    name: str = "WikipediaSearchTool"
    description: str = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ua = UserAgent()
        custom_user_agent = ua.random
        wikipedia = WikipediaAPIWrapper()
        return wikipedia.run(query)

In [35]:
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    tools=[
        DuckDuckGoSearchTool(),
        WikipediaSearchTool(),
    ],
)


In [36]:
prompt = "amazon 회사에 대해서 알려줘."

agent.invoke(prompt)



> Entering new AgentExecutor chain...
Thought:I need to find information about Amazon.  I can use Wikipedia to get a summary.

Action:
```json
{
  "action": "WikipediaSearchTool",
  "action_input": {
    "query": "Amazon"
  }
}
```


c:\workdir\github-space-cap\GPT-study-blog\fullstack-gpt\env\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\workdir\github-space-cap\GPT-study-blog\fullstack-gpt\env\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Amazon (company)
Summary: Amazon.com, Inc., doing business as Amazon (, AM-ə-zon; UK also , AM-ə-zən), is an American multinational technology company engaged in e-commerce, cloud computing, online advertising, digital streaming, and artificial intelligence. Founded in 1994 by Jeff Bezos in Bellevue, Washington, the company originally started as an online marketplace for books but gradually expanded its offerings to include a wide range of product categories, referred to as "The Everything Store". Today, Amazon is considered one of the Big Five American technology companies, the other four being Alphabet, Apple, Meta, and Microsoft. 
The company has multiple subsidiaries, including Amazon Web Services, providing cloud computing; Zoox, a self-driving car division; Kuiper Systems, a satellite Internet provider; and Amazon Lab126, a computer hardware R&D provider. Other subsidiaries include Ring, Twitch, IMDb, and Whole Foods Market. Its acquisition of Whole Foods in A

{'input': 'amazon 회사에 대해서 알려줘.',
 'output': "Amazon.com, Inc. is an American multinational technology company.  It's involved in e-commerce, cloud computing (AWS), online advertising, digital streaming, and artificial intelligence. Founded in 1994 by Jeff Bezos, it started as an online bookstore but has expanded significantly.  It's one of the Big Five American technology companies (along with Alphabet, Apple, Meta, and Microsoft) and is known for its wide range of products and services, including Amazon Prime, Kindle e-readers, and Alexa devices.  It also owns subsidiaries like Whole Foods Market, Twitch, and IMDb.  While a major success, Amazon has faced criticism regarding its labor practices, tax avoidance, and anti-competitive behavior."}